# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [38]:
# import libraries
import re
import pickle
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.base import BaseEstimator, TransformerMixin

import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings('ignore', category=UndefinedMetricWarning)

In [2]:
# load data from database
table_name='DisasterResponse'
con='sqlite:///./data/DisasterResponse.db'
df = pd.read_sql_table(table_name=table_name, 
                  con=con)
print(df.shape)
X = df['message']
y = df.iloc[:,4:]
category_names = y.columns

(26216, 40)


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
], verbose=10)

### 4. Train pipeline
- Split data into train and test sets

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

- Train pipeline

In [6]:
%%time
pipeline.fit(X_train, y_train)

[Pipeline] .............. (step 1 of 3) Processing vect, total=   5.9s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total= 4.1min
CPU times: user 4min 11s, sys: 667 ms, total: 4min 12s
Wall time: 4min 12s


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f93f7a50ee0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))],
         verbose=10)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)

In [8]:
for i in range(y_test.shape[1]):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.values[:,i], 
                                y_pred[:,i],
                                target_names=[f'class {i}' for i in range(y_test.iloc[:,i].nunique())]),
          '...................................................')

related :
              precision    recall  f1-score   support

     class 0       0.75      0.27      0.40      1542
     class 1       0.81      0.97      0.88      4972
     class 2       0.78      0.17      0.29        40

    accuracy                           0.80      6554
   macro avg       0.78      0.47      0.52      6554
weighted avg       0.79      0.80      0.76      6554
 ...................................................
request :
              precision    recall  f1-score   support

     class 0       0.90      0.99      0.94      5454
     class 1       0.89      0.44      0.59      1100

    accuracy                           0.90      6554
   macro avg       0.89      0.72      0.77      6554
weighted avg       0.90      0.90      0.88      6554
 ...................................................
offer :
              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00      6534
     class 1       0.00      0.00      0.00        

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7f93f7a50ee0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': 10,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7f93f7a50ee0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__us

In [10]:
parameters = {
    'clf__estimator__n_estimators': [100, 150],
    'clf__estimator__min_samples_split': [2, 3],
    
}

cv_clf = GridSearchCV(pipeline, param_grid=parameters, cv=5, verbose=3)

In [11]:
%%time
cv_clf.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[Pipeline] .............. (step 1 of 3) Processing vect, total=   3.7s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total= 2.9min
[CV 1/5] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100;, score=0.234 total time= 3.1min
[Pipeline] .............. (step 1 of 3) Processing vect, total=   3.6s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total= 2.9min
[CV 2/5] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100;, score=0.231 total time= 3.1min
[Pipeline] .............. (step 1 of 3) Processing vect, total=   3.7s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total= 3.0min
[CV 3/5] END clf__estimator__min_samples_split=2, clf__estimator__n_e

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f93f7a50ee0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))],
                                verbose=10),
             param_grid={'clf__estimator__min_samples_split': [2, 3],
                         'clf__estimator__n_estimators': [100, 150]},
             verbose=3)

In [12]:
cv_clf.cv_results_

{'mean_fit_time': array([180.67876115, 273.30802183, 167.08652864, 250.65577874]),
 'std_fit_time': array([2.42469698, 3.76445564, 1.05271272, 2.20909948]),
 'mean_score_time': array([ 6.9606823 ,  9.96037064,  7.1090744 , 10.11291709]),
 'std_score_time': array([0.05353495, 0.19623024, 0.11997391, 0.07842856]),
 'param_clf__estimator__min_samples_split': masked_array(data=[2, 2, 3, 3],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_clf__estimator__n_estimators': masked_array(data=[100, 150, 100, 150],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'clf__estimator__min_samples_split': 2,
   'clf__estimator__n_estimators': 100},
  {'clf__estimator__min_samples_split': 2,
   'clf__estimator__n_estimators': 150},
  {'clf__estimator__min_samples_split': 3,
   'clf__estimator__n_estimators': 100},
  {'clf__estimator__min_samples_split': 3,
   'clf__estimator__n_es

In [13]:
print(cv_clf.best_params_)

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 150}


In [14]:
cv_clf_optimised = cv_clf.best_estimator_
print(cv_clf_optimised)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f93f7a50ee0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=150)))],
         verbose=10)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
y_pred = cv_clf_optimised.predict(X_test)

for i in range(y_test.shape[1]):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.values[:,i], 
                                y_pred[:,i],
                                target_names=[f'class {i}' for i in range(y_test.iloc[:,i].nunique())]),
          '...................................................')

related :
              precision    recall  f1-score   support

     class 0       0.75      0.26      0.39      1542
     class 1       0.81      0.97      0.88      4972
     class 2       0.75      0.23      0.35        40

    accuracy                           0.80      6554
   macro avg       0.77      0.49      0.54      6554
weighted avg       0.79      0.80      0.76      6554
 ...................................................
request :
              precision    recall  f1-score   support

     class 0       0.90      0.99      0.94      5454
     class 1       0.89      0.43      0.58      1100

    accuracy                           0.90      6554
   macro avg       0.89      0.71      0.76      6554
weighted avg       0.89      0.90      0.88      6554
 ...................................................
offer :
              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00      6534
     class 1       0.00      0.00      0.00        

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('ada', MultiOutputClassifier(AdaBoostClassifier()))
], verbose=10)

In [17]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7f93f7a50ee0>)),
  ('tfidf', TfidfTransformer()),
  ('ada', MultiOutputClassifier(estimator=AdaBoostClassifier()))],
 'verbose': 10,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7f93f7a50ee0>),
 'tfidf': TfidfTransformer(),
 'ada': MultiOutputClassifier(estimator=AdaBoostClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_idf': 

In [18]:
parameters = {
    'ada__estimator__learning_rate': [0.1, 0.5, 1.0],
    'ada__estimator__n_estimators': [50,100],
    
}

cv_ada = GridSearchCV(pipeline, param_grid=parameters, cv=5, verbose=3)

In [19]:
%%time
cv_ada.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[Pipeline] .............. (step 1 of 3) Processing vect, total=   4.0s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing ada, total= 1.0min
[CV 1/5] END ada__estimator__learning_rate=0.1, ada__estimator__n_estimators=50;, score=0.200 total time= 1.1min
[Pipeline] .............. (step 1 of 3) Processing vect, total=   3.8s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing ada, total= 1.0min
[CV 2/5] END ada__estimator__learning_rate=0.1, ada__estimator__n_estimators=50;, score=0.199 total time= 1.1min
[Pipeline] .............. (step 1 of 3) Processing vect, total=   3.7s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing ada, total= 1.0min
[CV 3/5] END ada__estimator__learning_rate=0.1, ada__estimator__n_estimator

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f93f7a50ee0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('ada',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))],
                                verbose=10),
             param_grid={'ada__estimator__learning_rate': [0.1, 0.5, 1.0],
                         'ada__estimator__n_estimators': [50, 100]},
             verbose=3)

In [20]:
cv_ada.cv_results_

{'mean_fit_time': array([ 65.64675188, 128.30243254,  66.80677309, 127.36386385,
         66.57789607, 129.00320997]),
 'std_fit_time': array([0.76474295, 0.90298104, 0.29057092, 1.06784595, 0.61921956,
        2.12516117]),
 'mean_score_time': array([2.14447069, 3.3945488 , 2.16722307, 3.31723127, 2.15563879,
        3.37234511]),
 'std_score_time': array([0.04103889, 0.03084868, 0.06168422, 0.02014575, 0.03776404,
        0.11057228]),
 'param_ada__estimator__learning_rate': masked_array(data=[0.1, 0.1, 0.5, 0.5, 1.0, 1.0],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_ada__estimator__n_estimators': masked_array(data=[50, 100, 50, 100, 50, 100],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'ada__estimator__learning_rate': 0.1,
   'ada__estimator__n_estimators': 50},
  {'ada__estimator__learning_rate': 0.1, 'ada__estimator__n_e

In [21]:
print(cv_ada.best_params_)

{'ada__estimator__learning_rate': 0.5, 'ada__estimator__n_estimators': 100}


In [22]:
cv_ada_optimised = cv_ada.best_estimator_
print(cv_ada_optimised)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f93f7a50ee0>)),
                ('tfidf', TfidfTransformer()),
                ('ada',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(learning_rate=0.5,
                                                                    n_estimators=100)))],
         verbose=10)


In [23]:
y_pred = cv_ada_optimised.predict(X_test)

for i in range(y_test.shape[1]):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.values[:,i], 
                                y_pred[:,i],
                                target_names=[f'class {i}' for i in range(y_test.iloc[:,i].nunique())]),
          '...................................................')

related :
              precision    recall  f1-score   support

     class 0       0.64      0.23      0.33      1542
     class 1       0.80      0.96      0.87      4972
     class 2       0.83      0.12      0.22        40

    accuracy                           0.78      6554
   macro avg       0.75      0.44      0.47      6554
weighted avg       0.76      0.78      0.74      6554
 ...................................................
request :
              precision    recall  f1-score   support

     class 0       0.91      0.98      0.94      5454
     class 1       0.82      0.52      0.64      1100

    accuracy                           0.90      6554
   macro avg       0.87      0.75      0.79      6554
weighted avg       0.90      0.90      0.89      6554
 ...................................................
offer :
              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00      6534
     class 1       0.20      0.05      0.08        

In [24]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [25]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
            ])),

            ('starting_verb', StartingVerbExtractor()),
        ])),

        ('ada', MultiOutputClassifier(AdaBoostClassifier())),
         ])

In [31]:
cv_ada_sv = GridSearchCV(pipeline, param_grid=parameters, verbose=3)

In [32]:
%%time
cv_ada_sv.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END ada__estimator__learning_rate=0.1, ada__estimator__n_estimators=50;, score=0.200 total time= 1.5min
[CV 2/5] END ada__estimator__learning_rate=0.1, ada__estimator__n_estimators=50;, score=0.199 total time= 1.5min
[CV 3/5] END ada__estimator__learning_rate=0.1, ada__estimator__n_estimators=50;, score=0.216 total time= 1.5min
[CV 4/5] END ada__estimator__learning_rate=0.1, ada__estimator__n_estimators=50;, score=0.204 total time= 1.5min
[CV 5/5] END ada__estimator__learning_rate=0.1, ada__estimator__n_estimators=50;, score=0.207 total time= 1.6min
[CV 1/5] END ada__estimator__learning_rate=0.1, ada__estimator__n_estimators=100;, score=0.205 total time= 2.6min
[CV 2/5] END ada__estimator__learning_rate=0.1, ada__estimator__n_estimators=100;, score=0.205 total time= 2.5min
[CV 3/5] END ada__estimator__learning_rate=0.1, ada__estimator__n_estimators=100;, score=0.223 total time=10.4min
[CV 4/5] END ada__estimator__lear

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x7f93f7a50ee0>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())])),
                                                                       ('starting_verb',
                                                                        StartingVerbExtractor())])),
                                       ('ada',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'ada__estimator__learning_rate': [0

In [ ]:
cv_ada_sv.cv_results_

{'mean_fit_time': array([ 83.16926956, 236.8550735 ,  81.79017577, 141.1135849 ,
         79.64691777, 139.01312203]),
 'std_fit_time': array([  1.84252464, 188.78445583,   1.25929942,   2.55382942,
          0.53254167,   1.79277991]),
 'mean_score_time': array([6.52313738, 7.63795733, 6.49769363, 7.49208102, 6.35461183,
        7.43567247]),
 'std_score_time': array([0.16897191, 0.17408468, 0.12994672, 0.08688975, 0.14582903,
        0.08221762]),
 'param_ada__estimator__learning_rate': masked_array(data=[0.1, 0.1, 0.5, 0.5, 1.0, 1.0],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_ada__estimator__n_estimators': masked_array(data=[50, 100, 50, 100, 50, 100],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'ada__estimator__learning_rate': 0.1,
   'ada__estimator__n_estimators': 50},
  {'ada__estimator__learning_rate': 0.1, 'ada__es

In [ ]:
print(cv_ada_sv.best_params_)

{'ada__estimator__learning_rate': 0.5, 'ada__estimator__n_estimators': 100}


In [ ]:
cv_ada_sv_optimised = cv_ada_sv.best_estimator_
print(cv_ada_sv_optimised)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x7f93f7a50ee0>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('starting_verb',
                                                 StartingVerbExtractor())])),
                ('ada',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(learning_rate=0.5,
                                                                    n_estimators=100)))])


In [ ]:
y_pred = cv_ada_sv_optimised.predict(X_test)

for i in range(y_test.shape[1]):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.values[:,i], 
                                y_pred[:,i],
                                target_names=[f'class {i}' for i in range(y_test.iloc[:,i].nunique())]),
          '...................................................')

related :
              precision    recall  f1-score   support

     class 0       0.64      0.23      0.33      1542
     class 1       0.80      0.96      0.87      4972
     class 2       0.71      0.12      0.21        40

    accuracy                           0.78      6554
   macro avg       0.71      0.44      0.47      6554
weighted avg       0.76      0.78      0.74      6554
 ...................................................
request :
              precision    recall  f1-score   support

     class 0       0.91      0.98      0.94      5454
     class 1       0.82      0.52      0.64      1100

    accuracy                           0.90      6554
   macro avg       0.87      0.75      0.79      6554
weighted avg       0.90      0.90      0.89      6554
 ...................................................
offer :
              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00      6534
     class 1       0.12      0.05      0.07        

### 9. Export your model as a pickle file

In [39]:
with open('classifier.pkl', 'wb') as file:
    pickle.dump(cv_clf_optimised, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.